In [1]:
from cos_nmf import *

In [3]:
# 데이터 파일 경로 설정
destination_file_path = '../Data/Prompting/dataset/destination_mood.csv'
perfume_file_path = '../Data/preprocess-data/final_perfume-info.csv'

destinations = pd.read_csv(destination_file_path)
perfumes = load_perfume_data(perfume_file_path)
    
# 여행지 분위기 추출
tfidf_vectorizer, destination_tfidf_matrix, destinations = extract_destination_moods(destinations, n_topics=18)
    
# 향수와 여행지 매핑
perfume_tfidf_matrix = map_perfumes_to_destinations(perfumes, tfidf_vectorizer)
    
target_destination_index = 3  # 예시로 첫 번째 여행지를 선택
recommended_perfumes = recommend_perfumes_for_destination(target_destination_index, destinations, destination_tfidf_matrix, perfume_tfidf_matrix, perfumes, top_n=10)
    
print(f"Recommended Perfumes for {destinations.loc[target_destination_index, 'city']}:")
print(recommended_perfumes)

Recommended Perfumes for Tokyo:
['Insolence Eau de Parfum Guerlain Guerlain', 'Fate for Women Amouage Amouage', 'Fancy Jessica Simpson Jessica Simpson', 'Wanted by Night Azzaro Azzaro', "Reb'l Fleur Rihanna Rihanna", 'CH Men Prive Carolina Herrera Carolina Herrera', 'Burberry Her Intense Burberry Burberry', 'Fantasy Britney Spears Britney Spears', 'Lalique White Lalique Lalique', 'Laudano Nero Tiziana Terenzi Tiziana Terenzi']


c:\Users\sohtk\anaconda3\Lib\site-packages\sklearn\decomposition\_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [3]:
print(len(recommended_perfumes))

10
